# Load packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# color-blind color scheme
plt.style.use('tableau-colorblind10')

## Load in old code

We build on the previous notebook:

In [ ]:
class LIF_neuron:
    # initialize a neuron class
    # provided parameter dictionary params
    def __init__(self, params):
        # attach parameters to object
        self.V_th, self.V_reset = params['V_th'], params['V_reset']   
        self.tau_m, self.g_L = params['tau_m'], params['g_L']        
        self.V_init, self.V_L = params['V_init'], params['V_L']       
        self.dt = params['dt']
        self.tau_ref = params['tau_ref']

        # initialize voltage and current
        self.v = 0.0
        # time steps since last spike
        self.refractory_counter = 0
    
    def LIF_step(self, I):
        """
            Perform one step of the LIF dynamics
        """
        
        currently_spiking = False
        
        if self.refractory_counter > 0:
            # if the neuron is still refractory
            self.v = self.V_reset
            self.refractory_counter = self.refractory_counter - 1
        elif self.v >= self.V_th:
            # if v is above threshold,
            # reset voltage and record spike event
            currently_spiking = True
            self.v = self.V_reset
            self.refractory_counter = self.tau_ref/self.dt
        else:
            # else, integrate the current:
            # calculate the increment of the membrane potential
            dv = self.voltage_dynamics(I)
            # update the membrane potential
            self.v = self.v + dv

        return self.v, currently_spiking
    
    def voltage_dynamics(self, I):
        """
            Calulcates one step of the LI dynamics
        """
        dv = (-(self.v-self.V_L) + I/self.g_L) * (self.dt/self.tau_m)
        return dv
        

In [ ]:
# define new class as child of old class
class ExpLIF_neuron(LIF_neuron):
    def __init__(self, params):
        # build on LIF neuron with same settings
        # (this will run __init__ of the parent class)
        super().__init__(params)
        
        # we only need to attach additional variables:
        self.DeltaT = params['DeltaT']
        self.V_exp_trigger = params['V_exp_trigger']
    
    # now we can just    
    def voltage_dynamics(self, I):
        """
            Calulcates one step of the exp-LI dynamics
        """
        dv = (-(self.v-self.V_L) + I/self.g_L + self.DeltaT * np.exp((self.v-self.V_exp_trigger)/self.DeltaT)) * (self.dt/self.tau_m)
        return dv
        

In [ ]:
params = {}
### typical neuron parameters###
params['V_th']    = -55. # spike threshold [mV]
params['V_reset'] = -75. #reset potential [mV]
params['tau_m']   = 10. # membrane time constant [ms]
params['g_L']     = 10. #leak conductance [nS]
params['V_init']  = -65. # initial potential [mV]
params['V_L']     = -75. #leak reversal potential [mV]
params['tau_ref']    = 2. # refractory time (ms)
params['dt'] = .1  # Simulation time step [ms]

# additional parameters for ExpLIF neurons
params['DeltaT'] = 10.0  # sharpness of exponential peak
params['V_exp_trigger'] = -55. # threshold for exponential depolarization [mV]
params['V_th'] = 0 # new reset threshold [mV]

# Extending to populations

In the last section, we worked with single neurons. But now, let's say we want to simulate a population of 100 neurons. We could use the same functions as above and hack something together:

In [ ]:
# population parameters
n_neurons = 100

population1 = [ExpLIF_neuron(params) for _ in range(n_neurons)]

In [ ]:
# # we now have a list of 100 neuron objects:
# population1

Let's give them all noisy currents:

In [ ]:
mean_I, std_I = 300, 100
n_steps = 10_000

# these will now become lists of lists (neurons, time steps)
voltages_arr = []
spikes_arr = []

for i, neuron in enumerate(population1):
    voltages = []
    spikes = []
    if i % 10 == 0:
        print(f"Working on neuron {i}")
    for _ in range(n_steps):
        I = np.random.normal(mean_I, std_I)
        v, s = neuron.LIF_step(I=I)
        voltages.append(v)
        spikes.append(s)
    voltages_arr.append(voltages.copy())
    spikes_arr.append(spikes.copy())

In [ ]:
# convert to numpy array
voltages_arr = np.array(voltages_arr)
spikes_arr = np.array(spikes_arr)

# convert spikes to spike_timings
spike_timings = [arr.nonzero()[0] for arr in spikes_arr]

for i in range(n_neurons):
    x = spike_timings[i]
    y = [i for _ in spike_timings[i]]
    plt.scatter(x, y, marker='.', c='black')
plt.xlim(9000,10000)
plt.xlabel('Time step')
plt.ylabel('# Neuron')
plt.show()

So it works, but that scales terribly, because every neuron is simulated sequentially.

How can we parallelize this?

## Resisting the urge to hack it

The essential issue is that `ExpLIF_neuron` only has scalar variables (`v`, `refractory_counter`, ...). We want to extend this to vectors, so that every neuron can be updated in parallel.

Having learned about `__super__`, we may be inclined to use it here, right? The issue is that we have baked in the single-neuron property into the class.

Remember: "a good architecture allows you to postpone decision making as long as possible".

So let's do things properly, and use populations as the base of our ExpLIF model.

All we need to do is change the scalar variables to vectors. The `if` statements can be neatly taken care of by Numpy indexing and slicing:

In [ ]:
class ExpLIF_population:
    def __init__(self, params):
        # attach parameters to object
        self.V_th, self.V_reset = params['V_th'], params['V_reset']   
        self.tau_m, self.g_L = params['tau_m'], params['g_L']        
        self.V_init, self.V_L = params['V_init'], params['V_L']       
        self.dt = params['dt']
        self.tau_ref = params['tau_ref']
        self.DeltaT = params['DeltaT']
        self.V_exp_trigger = params['V_exp_trigger']
        
        # number of neurons
        self.n_neurons = params["n_neurons"]

        # initialize voltages
        self.v = np.zeros(self.n_neurons)
        # time steps since last spike
        self.refractory_counter = np.zeros(self.n_neurons)
            
    def LIF_step(self, I):
        """
            Perform one step of the LIF dynamics
        """
        
        currently_spiking = np.array([False for _ in range(self.n_neurons)])
        
        # This is where the magic happens: numpy indexing.
        # first, we need to get the indices of the neurons
        # which are refractory, above threshold or neither:
        idx_ref = np.where(self.refractory_counter > 0)[0]
        idx_spk = np.where(self.v > self.V_th)[0]
        idx_else = np.where((self.refractory_counter <= 0) & (self.v <= self.V_th))[0]
        
        # if the neuron is still refractory
        self.v[idx_ref] = self.V_reset
        self.refractory_counter[idx_ref] -= 1
        
        # if v is above threshold,
        # reset voltage and record spike event
        currently_spiking[idx_spk] = True
        self.v[idx_spk] = self.V_reset
        self.refractory_counter[idx_spk] = self.tau_ref/self.dt
        
        # calculate the increment of the membrane potential
        dv = self.voltage_dynamics(I)
        # update the membrane potential only for non-spiking neurons
        self.v[idx_else] += dv[idx_else]

        return self.v, currently_spiking
        
    def voltage_dynamics(self, I):
        """
            Calulcates one step of the exp-LI dynamics
        """
        # Fortunately, this code already enabled vectors, due to numpy magic.
        dv = (-(self.v-self.V_L) + I/self.g_L + self.DeltaT * np.exp((self.v-self.V_exp_trigger)/self.DeltaT)) * (self.dt/self.tau_m)
        return dv
        

# Intermezzo: Mutability & a common trap

Let's say we now dare to simulate two neurons. We can just take our population class with n_neurons = 2:

In [ ]:
# population parameters
n_neurons = 2
params["n_neurons"] = n_neurons

population2 = ExpLIF_population(params)

In [ ]:
mean_I, std_I = 300, 100
n_steps = 100

# these will now become lists of arrays with shape (time steps, neurons)
voltages_arr = []
spikes_arr = []

for _ in range(n_steps):
    I = np.random.normal(mean_I, std_I, size=n_neurons)
    v, s = population2.LIF_step(I=I)
    voltages_arr.append(v)
    spikes_arr.append(s)

In [ ]:
plt.plot(voltages_arr)
plt.show()

Hmm? Why are the voltages constant?

We can simplify the operations to see more clearly what is happening:

In [ ]:
arr = []
v = np.array([0])          # stand-in for the voltages of the two neurons
for i in range(10):
    v += np.array([i])     # stand-in for the v = v + dv operation
    arr.append(v)
arr                          # this should be [0,1,3,6,10,...]

We can illustrate more clearly what is happening here by an even simpler example:

In [ ]:
a = [1,2]
b = a
a[0] = 2
b

So by modifying `a`, we have also modified `b`. This is because lists in Python are **mutable**: they are references to objects in memory, and if the object changes, so does the reference.

Integers on the other hand are immutable:

In [ ]:
a = 1
b = a
a = 2
b

So whether an object is mutable or immutable depends on the object type!

**This is a very common source of bugs.** (at least in my code...)

Forgetting that you are creating a reference instead of a new, independent object can lead to exactly the issues we saw above.

See https://realpython.com/python-mutable-vs-immutable-types/#mutability-vs-immutability for more information on types and mutability.

Now, how can we fix the above simulation? Give it a try (or a Google search):

In [ ]:
mean_I, std_I = 300, 100
n_steps = 100

# these will now become lists of arrays with shape (time steps, neurons)
voltages_arr = []
spikes_arr = []

for _ in range(n_steps):
    I = np.random.normal(mean_I, std_I, size=n_neurons)
    v, s = population2.LIF_step(I=I)    # propose your solution here
    voltages_arr.append(v)              # and/or here
    spikes_arr.append(s)                # and/or here


In [ ]:
plt.plot(voltages_arr)
plt.show()

## Simulating a larger population and plotting a spike raster

In [ ]:
# population parameters
n_neurons = 100
params["n_neurons"] = n_neurons

population3 = ExpLIF_population(params)

In [ ]:
mean_I, std_I = 300, 100
n_steps = 10_000

# these will now become lists of lists (neurons, time steps)
voltages_arr = []
spikes_arr = []

for _ in range(n_steps):
    I = np.random.normal(mean_I, std_I, size=n_neurons)
    v, s = population3.LIF_step(I=I)
    voltages_arr.append(v.copy())
    spikes_arr.append(s.copy())

In [ ]:
# convert to numpy array
voltages_arr = np.array(voltages_arr)
spikes_arr = np.array(spikes_arr)

x_range = (9000,10_000)
for i in range(n_neurons):
    spike_times = spikes_arr[x_range[0]:x_range[1],i].nonzero()[0]
    plt.scatter(spike_times + x_range[0], i*np.ones_like(spike_times), marker='.', c='black')
plt.xlabel('Time step')
plt.ylabel('# Neuron')
plt.show()